In [1]:
from enhance_ocod.labelling.ner_spans import lfs
from enhance_ocod.labelling.weak_labelling import (process_dataframe_batch, create_flat_tag, 
create_commercial_park_tag, remove_overlapping_spans, get_overlap_stats, remove_zero_length_spans)

import pandas as pd 


gt_path = 'data/enhanced_ocod_data_and_gt/ground_truth_dev_set_labels.csv'

gt_path = "data/training_data/ground_truth_test_set_labels.csv"

In [ ]:
df = pd.read_csv(gt_path)
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:273: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:281: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(


In [3]:
test = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)
                           
remove_zero_length_spans(test)
remove_overlapping_spans(test)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


Processing complete:
  - Processed 1000 rows successfully
  - Found 5016 total spans
  - Average 5.02 spans per row


In [4]:
get_overlap_stats(test)

{'total_rows': 1000,
 'rows_with_overlaps': 0,
 'overlap_percentage': 0.0,
 'total_spans_before': 4161,
 'total_overlaps_removed': 0,
 'spans_after': 4161}

In [8]:
test[0:3]

[{'text': '90 craven park, london (nw10 8qe)',
  'spans': [{'text': '90', 'start': 0, 'end': 2, 'label': 'STREET_NUMBER'},
   {'text': 'craven park', 'start': 3, 'end': 14, 'label': 'STREET'},
   {'text': 'london', 'start': 16, 'end': 22, 'label': 'CITY'},
   {'text': 'nw10 8qe', 'start': 24, 'end': 32, 'label': 'POSTCODE'}],
  'row_id': 0},
 {'text': 'land on the west side of wisbech road, march',
  'spans': [{'text': 'wisbech road',
    'start': 25,
    'end': 37,
    'label': 'STREET'},
   {'text': 'march', 'start': 39, 'end': 44, 'label': 'CITY'}],
  'row_id': 1},
 {'text': '20a bute street, london (sw7 3ex)',
  'spans': [{'text': '20', 'start': 0, 'end': 2, 'label': 'STREET_NUMBER'},
   {'text': 'bute street', 'start': 4, 'end': 15, 'label': 'STREET'},
   {'text': 'london', 'start': 17, 'end': 23, 'label': 'CITY'},
   {'text': 'sw7 3ex', 'start': 25, 'end': 32, 'label': 'POSTCODE'}],
  'row_id': 2}]

In [5]:
from enhance_ocod.training import convert_weak_labels_to_standard_format, evaluate_weak_labels


In [7]:
test2 = convert_weak_labels_to_standard_format(test)

overall_results, class_df = evaluate_weak_labels(test2, 
    "data/training_data/ner_ready/ground_truth_test_set_labels.json", 
    "enhance_ocod/data/model_performance", 
    'regex_labels')

Evaluating noisy predictions on regex_labels set...
Loaded 1000 ground truth examples
Loaded 1000 predictions


ValueError: Found input variables with inconsistent numbers of samples:
[4, 4, 6, 4, 3, 3, 4, 3, 6, 7, 5, 5, 6, 7, 4, 4, 2, 3, 6, 7, 4, 4, 7, 4, 3, 4, 3, 3, 9, 7, 4, 7, 5, 6, 9, 5, 4, 4, 3, 4, 6, 7, 4, 3, 4, 3, 4, 4, 4, 3, 4, 7, 5, 7, 3, 4, 6, 4, 5, 7, 4, 5, 4, 4, 4, 4, 5, 6, 4, 4, 4, 5, 5, 4, 7, 4, 3, 6, 6, 4, 3, 5, 5, 3, 5, 3, 5, 5, 4, 4, 6, 4, 4, 4, 4, 4, 5, 3, 4, 4, 5, 4, 4, 5, 5, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 3, 6, 5, 4, 4, 5, 4, 6, 7, 6, 6, 3, 4, 4, 4, 5, 4, 4, 4, 4, 6, 7, 4, 4, 7, 4, 7, 6, 2, 7, 5, 4, 6, 5, 5, 3, 3, 4, 5, 3, 4, 7, 6, 5, 5, 6, 4, 4, 4, 6, 5, 11, 3, 3, 6, 3, 4, 7, 6, 6, 4, 6, 7, 6, 4, 7, 5, 4, 4, 4, 7, 3, 6, 5, 4, 6, 7, 5, 4, 4, 3, 7, 6, 4, 6, 6, 6, 4, 4, 6, 4, 2, 4, 4, 9, 4, 7, 4, 4, 6, 4, 6, 7, 5, 4, 4, 5, 4, 7, 4, 6, 4, 6, 2, 6, 3, 3, 4, 4, 4, 7, 4, 7, 3, 6, 4, 5, 5, 4, 5, 3, 4, 5, 4, 6, 4, 4, 4, 6, 6, 5, 6, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 6, 5, 4, 4, 4, 5, 4, 3, 4, 3, 4, 2, 2, 4, 3, 5, 7, 6, 3, 7, 3, 7, 6, 6, 4, 4, 3, 4, 4, 4, 5, 6, 5, 4, 6, 6, 6, 5, 6, 6, 6,